# Data Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
# from sklearn.utils import resample

train_df = pd.read_feather("/content/drive/MyDrive/data_ftr/train_data.ftr")
# test_df = pd.read_feather("")

# Data Check

In [20]:
train_df.shape

(5531451, 191)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 191 entries, customer_ID to target
dtypes: category(11), datetime64[ns](1), float16(177), int64(1), object(1)
memory usage: 2.0+ GB


In [22]:
train_df = train_df.groupby('customer_ID')
train_df = train_df.tail(1)

In [23]:
train_df.shape

(458913, 191)

## Removing Non-Meaningful Data
- customer_id : 고객 식별 번호
- S_2 : 날짜

> customer_id와 S_2는 신용 데이터를 분석하는 데 있어서 유의미 하지 않은 데이터이므로 제거합니다.

In [24]:
# Customer_ID, S_2 컬럼 제거 (유의미 하지 않음)
train_df = train_df.drop(['customer_ID', 'S_2'], axis=1)

# Missing Value

In [25]:
# 각 컬럼별 결측치 확인
train_df.isnull().sum()

P_2         2969
D_39           0
B_1            0
B_2           31
R_1            0
           ...  
D_142     378598
D_143       2830
D_144          0
D_145       2830
target         0
Length: 189, dtype: int64

# Libarary Import

# Dataset Description

In [26]:
total_cols = train_df.columns.to_list()

# 수치형과 범주형 컬럼 구분
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# 전체 데이터 - cat_columns
num_columns = [col for col in total_cols if col not in cat_columns]

In [27]:
print(f'수치형 컬럼: {num_columns}')
print(f'범주형 컬럼: {cat_columns}')

수치형 컬럼: ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S

## Data Copy

In [28]:
num_columns_copy = num_columns.copy() # 수치형 데이터 복제
cat_columns_copy = cat_columns.copy() # 범주형 데이터 복제

# Handling missing values
> Replace categorical data with LabelEncoding

In [29]:
# 범주형 데이터는 전체 데이터를 one-hot인코딩 대체

for column in cat_columns:
    ohe = OneHotEncoder()
    train_df[column] = ohe.fit_transform(train_df[column].values.reshape(-1, 1)).toarray()
    print(f"{column} OneHotEncoding 완료")

B_30 OneHotEncoding 완료
B_38 OneHotEncoding 완료
D_114 OneHotEncoding 완료
D_116 OneHotEncoding 완료
D_117 OneHotEncoding 완료
D_120 OneHotEncoding 완료
D_126 OneHotEncoding 완료
D_63 OneHotEncoding 완료
D_64 OneHotEncoding 완료
D_66 OneHotEncoding 완료
D_68 OneHotEncoding 완료


In [30]:
for column in num_columns_copy:
    if train_df[column].isnull().sum() / train_df.shape[0] * 100 < 15:
        train_df[column].fillna(train_df[column].median(), inplace=True)
        print(f"{column} 결측치 중앙값 대체 완료")
    else:
        print(f"{column} 결측치 없음")

P_2 결측치 중앙값 대체 완료
D_39 결측치 중앙값 대체 완료
B_1 결측치 중앙값 대체 완료
B_2 결측치 중앙값 대체 완료
R_1 결측치 중앙값 대체 완료
S_3 결측치 없음
D_41 결측치 중앙값 대체 완료
B_3 결측치 중앙값 대체 완료
D_42 결측치 없음
D_43 결측치 없음
D_44 결측치 중앙값 대체 완료
B_4 결측치 중앙값 대체 완료
D_45 결측치 중앙값 대체 완료
B_5 결측치 중앙값 대체 완료
R_2 결측치 중앙값 대체 완료
D_46 결측치 없음
D_47 결측치 중앙값 대체 완료
D_48 결측치 중앙값 대체 완료
D_49 결측치 없음
B_6 결측치 중앙값 대체 완료
B_7 결측치 중앙값 대체 완료
B_8 결측치 중앙값 대체 완료
D_50 결측치 없음
D_51 결측치 중앙값 대체 완료
B_9 결측치 중앙값 대체 완료
R_3 결측치 중앙값 대체 완료
D_52 결측치 중앙값 대체 완료
P_3 결측치 중앙값 대체 완료
B_10 결측치 중앙값 대체 완료
D_53 결측치 없음
S_5 결측치 중앙값 대체 완료
B_11 결측치 중앙값 대체 완료
S_6 결측치 중앙값 대체 완료
D_54 결측치 중앙값 대체 완료
R_4 결측치 중앙값 대체 완료
S_7 결측치 없음
B_12 결측치 중앙값 대체 완료
S_8 결측치 중앙값 대체 완료
D_55 결측치 중앙값 대체 완료
D_56 결측치 없음
B_13 결측치 중앙값 대체 완료
R_5 결측치 중앙값 대체 완료
D_58 결측치 중앙값 대체 완료
S_9 결측치 없음
B_14 결측치 중앙값 대체 완료
D_59 결측치 중앙값 대체 완료
D_60 결측치 중앙값 대체 완료
D_61 결측치 중앙값 대체 완료
B_15 결측치 중앙값 대체 완료
S_11 결측치 중앙값 대체 완료
D_62 결측치 중앙값 대체 완료
D_65 결측치 중앙값 대체 완료
B_16 결측치 중앙값 대체 완료
B_17 결측치 없음
B_18 결측치 중앙값 대체 완료
B_19 결측치 중앙값 대체 완료
B_20 결측치 중앙값 대체 완료
S_12 결측치 중앙값 대체 

In [31]:
# train_df를 cat_columns와 num_columns를 합체
train_df = pd.concat([train_df[cat_columns_copy], train_df[num_columns_copy]], axis=1)

In [32]:
train_df.head(10)

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
12,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.007187,0.004234,0.005085,NaN,0.005810,0.002970,0.008530,0
25,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.002981,0.007481,0.007874,NaN,0.003284,0.003170,0.008514,0
38,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.007381,0.006622,0.000965,NaN,0.002201,0.000834,0.003445,0
51,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.002705,0.006184,0.001899,NaN,0.008186,0.005558,0.002983,0
64,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008156,0.006943,0.000905,0
77,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.009399,0.007835,0.000957,NaN,0.009056,0.006943,0.001446,0
90,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,0.006012,0.005222,0.000937,NaN,0.009789,0.003704,0.006275,0
103,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.002262,0.005219,0.002153,NaN,0.001169,0.003349,0.000675,0
116,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,1.000977,0.007061,0.875977,0.18457,1.004883,0.003349,0.734375,1
129,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.009811,0.002213,0.004070,NaN,0.005428,0.007107,0.006271,1


# Model Training

In [33]:
X = train_df.drop('target', axis=1)
y = train_df['target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train.shape

(367130, 188)

In [35]:
X_test.shape

(91783, 188)

In [36]:
y_train.shape

(367130,)

In [37]:
y_test.shape

(91783,)

## XGBoost

In [38]:
# XGBoost 모델 학습 (eval_set을 사용하여 검증 데이터를 모니터링)
model = XGBClassifier(n_estimators=200, learning_rate=0.15, subsample=0.5)
model.fit(X_train, y_train)

# pkl 파일 추출
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [39]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()

    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

# 예측 확률 계산
pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = pd.DataFrame({'prediction': pred_prob}, index=X_test.index)
y_true = pd.DataFrame({'target': y_test}, index=X_test.index)

# 성능 평가
print("AMEX Score:", amex_metric(y_true, y_pred))

AMEX Score: 0.7773397985279838


In [40]:
# customer_ID와 prediction을 submission.csv로 추출
submission = pd.DataFrame({'customer_ID': X_test.index, 'prediction': y_pred.values.flatten()})
submission.head(30)

,customer_ID,prediction
0,403008,0.026179
1,4224637,0.630562
2,444377,0.065996
3,5363086,0.000738
4,211489,0.001775
5,1248043,0.000971
6,3497151,0.000657
7,5394574,0.075946
8,1435073,0.002304
9,3650300,0.714861


In [41]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델의 성능 리포트를 출력하는 함수
def generate_model_report(y_true, y_pred):
    # 평가 지표 계산
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # 결과 출력
    print("Model Performance:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}")

# 예측 결과와 실제 타겟 데이터를 준비
y_true = y_test
y_pred = model.predict(X_test)

# 모델의 성능 리포트 출력
generate_model_report(y_true, y_pred)

Model Performance:
MSE: 0.10130416308030898
MAE: 0.10130416308030898
R^2: 0.4688086798250328


In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)

# 정밀도 계산
precision = precision_score(y_test, y_pred)

# 재현율 계산
recall = recall_score(y_test, y_pred)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# 분류 보고서 출력
report = classification_report(y_test, y_pred)
print("\nXGBoost Report:\n", report)

Accuracy: 0.898695836919691
Precision: 0.8021465235650956
Precision: 0.8021465235650956
Recall: 0.8031686700930213
F1 Score: 0.8026572714152305

XGBoost Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93     68240
           1       0.80      0.80      0.80     23543

    accuracy                           0.90     91783
   macro avg       0.87      0.87      0.87     91783
weighted avg       0.90      0.90      0.90     91783



# test data

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

In [44]:
# test_data 로드
test_df = pd.read_feather("/content/drive/MyDrive/data_ftr/test_data.ftr")

In [45]:
test_df = test_df.groupby('customer_ID')
test_df = test_df.tail(1)
test_df.shape

(924621, 190)

In [46]:
# test_df 범주형 데이터는 전체 데이터를 LabelEncoding으로 대체(우선 범주형, 수치형 데이터 분리) -> LabelEncoding 후 다시 합치기
total_cols = test_df.columns.to_list()

# 수치형과 범주형 컬럼 구분
cat_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
# 전체 데이터 - cat_columns
num_columns = [col for col in total_cols if col not in cat_columns]

# 범주형 데이터는 전체 데이터를 onehot
for column in cat_columns:
    ohe = OneHotEncoder()
    test_df[column] = ohe.fit_transform(test_df[column].values.reshape(-1, 1)).toarray()
    print(f"{column} OneHotEncoding 완료")

for column in num_columns_copy:
    if train_df[column].isnull().sum() / train_df.shape[0] * 100 < 15:
        train_df[column].fillna(train_df[column].median(), inplace=True)
        print(f"{column} 결측치 중앙값 대체 완료")
    else:
        print(f"{column} 결측치 없음")

# test_df를 cat_columns와 num_columns를 합체
test_df = pd.concat([test_df[cat_columns], test_df[num_columns]], axis=1)

B_30 OneHotEncoding 완료
B_38 OneHotEncoding 완료
D_114 OneHotEncoding 완료
D_116 OneHotEncoding 완료
D_117 OneHotEncoding 완료
D_120 OneHotEncoding 완료
D_126 OneHotEncoding 완료
D_63 OneHotEncoding 완료
D_64 OneHotEncoding 완료
D_66 OneHotEncoding 완료
D_68 OneHotEncoding 완료
P_2 결측치 중앙값 대체 완료
D_39 결측치 중앙값 대체 완료
B_1 결측치 중앙값 대체 완료
B_2 결측치 중앙값 대체 완료
R_1 결측치 중앙값 대체 완료
S_3 결측치 없음
D_41 결측치 중앙값 대체 완료
B_3 결측치 중앙값 대체 완료
D_42 결측치 없음
D_43 결측치 없음
D_44 결측치 중앙값 대체 완료
B_4 결측치 중앙값 대체 완료
D_45 결측치 중앙값 대체 완료
B_5 결측치 중앙값 대체 완료
R_2 결측치 중앙값 대체 완료
D_46 결측치 없음
D_47 결측치 중앙값 대체 완료
D_48 결측치 중앙값 대체 완료
D_49 결측치 없음
B_6 결측치 중앙값 대체 완료
B_7 결측치 중앙값 대체 완료
B_8 결측치 중앙값 대체 완료
D_50 결측치 없음
D_51 결측치 중앙값 대체 완료
B_9 결측치 중앙값 대체 완료
R_3 결측치 중앙값 대체 완료
D_52 결측치 중앙값 대체 완료
P_3 결측치 중앙값 대체 완료
B_10 결측치 중앙값 대체 완료
D_53 결측치 없음
S_5 결측치 중앙값 대체 완료
B_11 결측치 중앙값 대체 완료
S_6 결측치 중앙값 대체 완료
D_54 결측치 중앙값 대체 완료
R_4 결측치 중앙값 대체 완료
S_7 결측치 없음
B_12 결측치 중앙값 대체 완료
S_8 결측치 중앙값 대체 완료
D_55 결측치 중앙값 대체 완료
D_56 결측치 없음
B_13 결측치 중앙값 대체 완료
R_5 결측치 중앙값 대체 완료
D_58 결측치 중앙값 대체 완료
S_9 결측치 없

In [47]:
# customer_ID 따로 저장
customer_ID = test_df['customer_ID']

In [48]:
# S_2, customer_ID 컬럼 드랍
test_df = test_df.drop(columns=['customer_ID', 'S_2'])

In [49]:
# XGBoostClassifier 모델로 예측
pred = model.predict_proba(test_df)[:, 1]

In [50]:
# customer_ID와 prediction을 submission.csv로 추출
submission = pd.DataFrame({'customer_ID': customer_ID, 'prediction': pred})
submission.to_csv('submission_XGBoost(OneHot&15%_Median).csv', index=False)